In [2]:
import mysql.connector
import pandas as pd

In [24]:
#connect to mysql servel
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Parvashah@1234",
  database="yelpdb"
)

print(mydb)

mycursor = mydb.cursor()


In [ ]:
# create business table
mycursor.execute("DROP TABLE IF EXISTS business")
mycursor.execute("CREATE TABLE business (business_id VARCHAR(255) PRIMARY KEY, name VARCHAR(255), address VARCHAR(255), city VARCHAR(255), state TEXT(2), postal_code INT(5), stars DECIMAL(10,1), review_count INT, is_open BOOL)")

In [25]:
# Insert data into business table from json file

# dataframe creation
df_b = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
# delete unwanted columns
del df_b['attributes'], df_b['hours'], df_b['categories'], df_b['latitude'], df_b['longitude']

sql = "INSERT IGNORE INTO business (business_id, name, address, city, state, postal_code, stars, review_count, is_open) VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s)"
val = list(df_b.itertuples(index=False,name=None))

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

150346 was inserted.


In [ ]:
# create review table
mycursor.execute("DROP TABLE IF EXISTS review")
mycursor.execute("CREATE TABLE review (review_id VARCHAR(255) PRIMARY KEY, user_id VARCHAR(255), business_id VARCHAR(255), stars INT(1), text TEXT(65535), date DATETIME)")

In [ ]:
# Insert data into review table from json file
# dataframe creation is done in chunks since it is a large file which cannot be read at once.
chunks = pd.read_json('yelp_dataset/yelp_academic_dataset_review.json', lines=True, chunksize = 10000)
reviews = pd.DataFrame()
for chunk in chunks:
    print(chunk)
    reviews = pd.concat([reviews, chunk])
# delete unwanted columns from dataframe
del reviews['useful'], reviews['funny'], reviews['cool']

val = list(reviews.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO review (review_id, user_id, business_id, stars, text, date) VALUES (%s, %s, %s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

In [ ]:
# create user table
mycursor.execute("DROP TABLE IF EXISTS user")
mycursor.execute("CREATE TABLE user (user_id VARCHAR(255) PRIMARY KEY, user_id VARCHAR(255), business_id VARCHAR(255), stars INT(1), text TEXT(65535), date DATETIME)")

In [3]:
chunks = pd.read_json('yelp_dataset/yelp_academic_dataset_user.json', lines=True, chunksize = 10000)
users = pd.DataFrame()
for chunk in chunks:
    print(chunk)
    reviews = pd.concat([users, chunk])

                     user_id    name  review_count        yelping_since  \
0     qVc8ODYU5SZjKXVBgXdI7w  Walker           585  2007-01-25 16:47:26   
1     j14WgRoU_-2ZE1aw1dXrJg  Daniel          4333  2009-01-25 04:35:42   
2     2WnXYQFK0hXEoTxPtV2zvg   Steph           665  2008-07-25 10:41:00   
3     SZDeASXq7o05mMNLshsdIA    Gwen           224  2005-11-29 04:38:33   
4     hA5lMy-EnncsH4JoR-hFGQ   Karen            79  2007-01-05 19:40:59   
...                      ...     ...           ...                  ...   
9995  Ry6R2TMTQOInMXJR2TJh4w   Allen             5  2010-07-17 20:19:51   
9996  UnHIN6t0MVRdC0Jo9oD9PQ  Shanel            43  2010-10-09 21:28:32   
9997  nbF6fWySgbbD15TGSOKN6Q  Britta            43  2010-09-19 00:13:03   
9998  rVuP6BdGt_8QF7PA0kRyvQ     Nel           392  2011-07-15 22:54:35   
9999  fPvAAXp-B-fTS3_HOxvqlg     Jon            76  2008-07-06 04:11:54   

      useful  funny   cool                                              elite  \
0       7217   125

                      user_id       name  review_count        yelping_since  \
20000  mgblDTuUe7MiHPW3hXREcg      David            25  2011-03-27 07:34:21   
20001  xMW9tsOdVdCxQSflkUiLog     Esther            19  2011-02-18 21:56:16   
20002  lQHIkpAMfGXirsWTmwBakA  Christiny             5  2010-09-23 18:58:56   
20003  kx3_7QkTlc5OjDgU2tlVqA    Whitney            87  2011-12-10 05:50:41   
20004  yQWHD-RgfiocahsakJRt1Q   Kristine            77  2008-02-01 18:36:44   
...                       ...        ...           ...                  ...   
29995  8IArCBeNftW8PjL3JOmA5g      Nicki           446  2012-09-11 19:45:03   
29996  gn4UQjtmsclNXeOzm2Sp5w        May            26  2011-10-08 15:49:09   
29997  YJRhX3dL8QdS2w326bG5QA     Hansel           604  2011-01-26 20:15:18   
29998  NUlSb7vDFic0azQ0W3ef7g   Jonathan            13  2010-11-23 01:56:47   
29999  YIxMH--BQgvxxcNMYD2uEA   Paulette            91  2011-10-26 00:38:10   

       useful  funny  cool                         

                      user_id     name  review_count        yelping_since  \
40000  G_JPTGnoSG0KHkR3N7Q0cw     Greg            27  2011-10-10 23:34:39   
40001  yyol3ymAowg_WBgfmbkdkg   Smaran            15  2013-05-06 02:21:31   
40002  CONvdTfeuJxmjabeoPcHIg  Griffin            28  2012-09-13 13:00:48   
40003  SGj5owE2WK_e9SSi45EzcA     Elyn            49  2011-04-01 17:04:55   
40004  GNSP-GWW3rOqmBfS0Uke0g    Aliza            17  2005-07-11 03:56:38   
...                       ...      ...           ...                  ...   
49995  K_sPBSw9NXANtRDnzj3L3A   Melody             9  2015-08-04 18:57:30   
49996  a4m4ZwQu_0zsJhTrAHBsdA    Chris           428  2014-01-17 03:32:21   
49997  vnJyZJ2GGZkGYtGABHqRVg   Alyssa            16  2014-07-18 20:51:17   
49998  zhx48x_XOjJQBA0DMnPG2Q  Rasheen            91  2014-07-07 15:08:13   
49999  btSaJqMiUwMemMZ1AhCe3Q  Cameron           333  2012-08-02 01:10:16   

       useful  funny  cool                      elite  \
40000      22     

                      user_id       name  review_count        yelping_since  \
60000  0b5hnIYWIDSl5NFXDG1CIQ      Gemma             8  2014-07-29 15:54:46   
60001  Vdxx8Qp8TqbDpwDgJwAkHA     Jackie             3  2015-04-28 22:19:52   
60002  lwzLDVShC60VWFIUNtvMjA       Luis            50  2013-06-13 23:01:18   
60003  U5oAPuwiWybziOIQ4w7Srw      Jerry            64  2010-12-30 00:58:10   
60004  BhdEDli35gQ9MNAmkSVc2g      Deedy            27  2014-08-24 00:15:09   
...                       ...        ...           ...                  ...   
69995  VaK7fDZQl9WaujeFiM8eeA  Tandekile             2  2018-05-12 23:02:05   
69996  v5vClPmXPONhQ5Y4rpbFtw    Brandon             1  2016-10-10 23:14:09   
69997  1EJhlCJs59Oa-5gKeR6cDA       Chad            51  2016-10-16 19:05:42   
69998  t_Ye5k8WuV_fpS6HQRKXdg      Erica             7  2011-12-28 21:44:42   
69999  93foJ4V84A-nd90MCt_ZIA      Trang             7  2013-07-02 21:54:50   

       useful  funny  cool elite  \
60000       2  

                      user_id     name  review_count        yelping_since  \
80000  03ly5igcjFVe7DFWj8E8sg   Justin            10  2014-06-30 02:21:54   
80001  upf_T2jkMS0tk98-PFEPnw   George             8  2014-03-11 20:03:44   
80002  KbHYruFdQvBSTEuXaMcl6Q   Mariah            26  2013-02-23 01:15:51   
80003  2ibyYOO6v0Don8pZ8xO11g    Irene             4  2011-03-21 13:12:00   
80004  gaMPz4Efb2uCdBLcS0oDBg     Dena            11  2013-05-29 20:39:24   
...                       ...      ...           ...                  ...   
89995  sQnEcOD-RIzXZwV4n-4Ztw  Katrine            10  2012-01-20 01:49:46   
89996  1duFSC3IrnnywjUnWKjl8Q  Corinne            11  2015-10-18 21:25:44   
89997  u84ugsSzoD_gb2bdlTRn2A     Esma             4  2018-03-09 18:21:26   
89998  _FHLJJ7QRV0M9ebveb-5_Q  Sabrina            15  2016-06-26 12:52:39   
89999  KUFTLo5NlzewLUPX-xBrZg  Yolanda           103  2014-08-21 16:04:19   

       useful  funny  cool                      elite  \
80000       9     

...

                       user_id      name  review_count        yelping_since  \
520000  Gp1oF65y8e7GMk1OJJv8qg    Blayne             5  2013-04-18 16:49:59   
520001  ALoNc_vcCOqGhZcPx4Dbcw  Karoline             5  2012-12-27 00:37:38   
520002  W5tMILZxjPPoVa_pUwvdbw  Virginia             1  2015-09-05 17:55:37   
520003  5xFlHM8gEvqslTun1SyQwQ    Nicole             1  2016-12-19 21:24:11   
520004  huGtp4jcPmIgmh2YjyINIw      Beth             4  2019-07-20 03:14:36   
...                        ...       ...           ...                  ...   
529995  LxzlH04a_PJpjXCqkkEBOw    Claire             4  2016-04-27 22:45:37   
529996  90be12_bmqOIBlJzOaiyZA     James            11  2012-09-04 03:18:32   
529997  Ry2JakFFsE-kX6hhXxRHCQ     Nelly            21  2016-02-05 04:23:59   
529998  FVD0InGQV4-DrUePwmLgzA   Gabriel             5  2015-10-20 15:46:39   
529999  GpScojHoJN3dcAExc3OO5w      Adam            10  2017-06-28 01:11:16   

        useful  funny  cool elite                 f

                       user_id       name  review_count        yelping_since  \
540000  0jf4NLjylJHY-VIUpTNEMw  Christina             3  2016-09-07 20:37:08   
540001  M5kGtLBNVvSHAWPq4NYMLA     Meghan             1  2013-10-23 17:53:35   
540002  DNFZ3PchGy99BZM9fYl7LA     Martha             1  2012-12-03 16:56:45   
540003  t-RX4zOCzFD-uinWq169mw     Justin             1  2020-01-22 01:27:46   
540004  AM8T7Gtp8zv7L66m4zywjA      Chris             1  2015-02-06 21:44:39   
...                        ...        ...           ...                  ...   
549995  RRaLI866g_oBCHTo4gsuLQ      Terry            19  2015-06-29 16:48:23   
549996  zZbWJHnvjQfdYNalcjMQTg     Lauren             7  2012-04-28 15:17:00   
549997  poExuBulIl5gFBrjosjVMg       Erin             1  2020-12-18 23:52:41   
549998  Hx3m10u4bjUAXEkhfXaLvg     Ronald             3  2013-02-18 15:24:34   
549999  GCA8uTAF_R5MramzWnlN0g  Roseville             2  2017-05-25 16:38:38   

        useful  funny  cool elite frien

                       user_id     name  review_count        yelping_since  \
560000  mDySrZG8qMql2weyK6nPpA        S             8  2012-10-30 21:08:57   
560001  -QiyFOEuZpqzNeeMl88g5w     Juli             5  2017-11-06 21:14:06   
560002  bD_1kwM4ZrTlhh1C5i_xFw    Jerae             9  2015-05-15 00:36:14   
560003  E5lZbGPULNLkh3ok4SREcg    Frank             4  2008-10-28 02:42:39   
560004  GEwia--xRx1XZ7y2wE7Zog  Heather             2  2016-11-28 06:49:34   
...                        ...      ...           ...                  ...   
569995  SXvGhPpRhNWZryOBK2usLw     Dean             4  2011-12-17 20:55:10   
569996  J7lS3zcxW1VvOliYRD_arw      Ms.             5  2018-01-12 18:20:46   
569997  xTzyZAKsTrQKhnDUYVSi0A      Don            27  2013-12-06 18:52:59   
569998  87WO0mTup878fv3FmB72Sw  Phillip             3  2015-05-06 21:25:13   
569999  mNDvW2zduN03eRoNXke72w    Sarah            16  2015-05-03 03:42:48   

        useful  funny  cool elite friends  fans  ...  complimen

                       user_id       name  review_count        yelping_since  \
580000  i5xp4LWigr7qlM1c466lYg       Ryan             1  2011-07-24 17:49:04   
580001  _oBjdX_eWnugi4I830z53A       Joan             4  2011-08-25 18:21:23   
580002  sgqXDCWVf1s8mJZmYOCTCQ  Elizabeth             3  2016-08-06 22:42:35   
580003  jQSH0UqrtAPjBCn641Qu8g      David             5  2018-08-02 00:58:19   
580004  uwzO2LhHAGpRhPd4umrvVg     Leslie             5  2018-03-15 15:33:43   
...                        ...        ...           ...                  ...   
589995  BGjnFNamF1LB41oxfMYMjw       Jeff            11  2015-07-16 14:57:27   
589996  LIbv6tSj74sk3jhwmQaj6Q    Patrick            12  2015-10-25 01:59:21   
589997  2XKjZuxhzxRUVSAK-Scclg      Aaron             5  2019-07-13 17:35:53   
589998  BTBNwSuihpER-uV0NKt3gg        Iiz            11  2009-10-08 19:24:50   
589999  5y1mwd2q081gc8YO8fuvqQ   Richelle             2  2019-06-05 01:35:17   

        useful  funny  cool elite frien

KeyboardInterrupt: 

In [ ]:
users.sample(10)